In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(num_results=2)
search_results = search.invoke("what is the weather in SF")

print(search_results)

tools = [search]

snippet: Light west southwest wind increasing to 6 to 11 mph in the morning. Friday Night. A slight chance of drizzle. Mostly cloudy, with a low around 58. West southwest wind 3 to 8 mph. Saturday. A chance of drizzle. Partly sunny, with a high near 73. South wind 5 to 9 mph becoming west in the afternoon., title: National Weather Service, link: https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA, snippet: NWS Forecast Office San Francisco, CA. Weather.gov > San Francisco Bay Area, CA Current Hazards. Current Outlooks; Daily Briefing; Submit Report; Detailed Hazards; Tsunami ... National Weather Service San Francisco Bay Area, CA 21 Grace Hopper Ave, Stop 5 Monterey, CA 93943-5505 (831) 656-1725 Comments? Questions? Please Contact Us. ..., title: San Francisco Bay Area, CA - National Weather Service, link: https://www.weather.gov/mtr/?os=vb...&ref=app


In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [8]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'duckduckgo_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_83GUlILQ2KPG8Mv82Wah1XyI', 'type': 'tool_call'}]


In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi!")]})

response["messages"]

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}, id='491686bb-6a44-424a-b383-2634a2381b2a'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 87, 'total_tokens': 97, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a20a4ee344', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf4b4727-a3e7-4037-a700-b986d4f88824-0', usage_metadata={'input_tokens': 87, 'output_tokens': 10, 'total_tokens': 97, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in SF?")]}
)

response["messages"]

[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='9f81e255-21d0-4653-a13f-4bc683a57faa'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ODFzQJE00nBLsM7YlbMRFe85', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 91, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a20a4ee344', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-91aa0160-a06d-408a-bdd3-5de6338754f9-0', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ODFzQJE00nBLsM7YlbMRFe85', 'type': 'tool_call'}], usage_metadata={'input_token

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YczwkE4heQAmcOQEVoWeWGWI', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 92, 'total_tokens': 112, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b4bd2fda-5089-4916-ab5a-fc9687e33034-0', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_YczwkE4heQAmcOQEVoWeWGWI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 20, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}


In [16]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: duckduckgo_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: duckduckgo_results_json
Tool output was: content='snippet: Current Hazards. Submit Report; Current Outlooks; Daily Briefing; Detailed Hazards; Tsunami; Graphical Hazardous Weather Outlook; Current Conditions. Observations; ... National Weather Service San Francisco Bay Area, CA 21 Grace Hopper Ave, Stop 5 Monterey, CA 93943-5505 (831) 656-1725 Comments? Questions? Please Contact Us. ..., title: San Francisco Bay Area, CA - National Weather Service, link: https://www.weather.gov/mtr/?os=a&ref=app, snippet: Current conditions at SAN FRANCISCO DOWNTOWN (SFOC1) Lat: 37.77056°NLon: 122.42694°WElev: 150.0ft. NA. 63°F. 17°C. ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 1:26 pm PDT Oct 11, 2024. ... National Weather Service; San Francisco Bay Area, CA; 21 Grace Hopper Ave, Stop 5; Monterey, CA 93943-5505; Comments? Questions?, title: National Weather Service, lin

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 89, 'total_tokens': 100, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-3415c70c-036f-4030-823a-72a0c20ce4cc-0', usage_metadata={'input_tokens': 89, 'output_tokens': 11, 'total_tokens': 100, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 112, 'total_tokens': 118, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b7da05a0-7ab7-42b7-9128-bcaf854f109c-0', usage_metadata={'input_tokens': 112, 'output_tokens': 6, 'total_tokens': 118, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----
